In [1]:
import requests
import re
import tldextract
from bs4 import BeautifulSoup
init_url='http://l-team.org/about-us.html'

In [2]:
def get_host(friend_links):
    sur_host={}
    fail_host={}
    problem_host={}
    #requests.adapters.DEFAULT_RETRIES = 5 # 增加重连次数
    s = requests.session()
    s.keep_alive = False
    s.adapters.DEFAULT_RETRIES = 10
    for key in friend_links.keys():
        
        try:
            #print(key,friend_links[key],s.get(friend_links[key]))
            if s.get(friend_links[key],timeout=10).status_code==200:
                sur_host[key]=friend_links[key]
            else:
                problem_host[key]=friend_links[key]
        except Exception as e:
            #print("{}{}".format(e,friend_links[key]))
            fail_host[key]=friend_links[key]
    return sur_host,problem_host,fail_host

In [3]:
#当友链是单独页面时，从首页提取友链页面链接，再提取友链
#存在网页解码识别不了“友情链接”问题https://www.sqlsec.com
def from_index_search_links(url):
    try:
        a_url=''
        s = requests.session()
        s.keep_alive = False
        s.adapters.DEFAULT_RETRIES = 10
        result=s.get(url,timeout=10)
        bsObj=BeautifulSoup(result.text,'html.parser')
        a_links=bsObj.find_all('a')
        for a_link in a_links:
            #print(a_link)
            #if '友链' in a_link.text or '友情链接' in a_link.text or 'FriendLink' in a_link.text or 'Friendlink' in a_link.text or 'friends' in a_link.text or 'links' in a_link.text or 'Friends' in a_link.text or 'Links' in a_link.text:
            if [True for str in ("友链","友情链接","FriendLink","Friendlink","friends","FRIENDS",'links','Links','Link')if str in a_link.text]:
                if 'http' not in a_link.get('href'):
                    a_url=url.rstrip('/')+a_link.get('href')
                else:
                    a_url=a_link.get('href')
            
        return a_url
    except Exception as err:
        #print(str(err))
        pass


def from_link_search_friend(link):
    friend_link_url={}
    try:    
        s = requests.session()
        s.keep_alive = False
        s.adapters.DEFAULT_RETRIES = 10
        result=s.get(link,timeout=10)
        bsObj=BeautifulSoup(result.text,'html.parser')
        a_urls = bsObj.find_all('a')
        for a_url in a_urls:
            if re.search('http(.+?)(com|net|blog|pw|cc|xyz|club|org|cn|io|info|me|im|link)(\/)?$',a_url.get('href')) != None and a_url.get('href') not in link:
                #print(a_url.get('href'),a_url.text)
                if a_url.get('href')!='http://www.52bug.cn':
                    friend_link_url[a_url.text.strip('\n')]=(a_url.get('href').strip('/'))
    except Exception as e:
        pass
        #print(str(e))
    #print(len(friend_link_url.keys()))
    return friend_link_url

#增加主页关键字"链接" or "Links" https://www.virzz.com
#http://iamstudy.cnblogs.com/
#需要增加关键字Bookmarks（https://www.melodia.pw/的section标签中）,Friendship website（http://poetichacker.com/的div标签中）
def search_link(key,url):
    result=[]
    friend_link_url={}
    try:
        html_result=requests.get(url)
        status=False
        #print(url)
        if [True for str in ("友链","友情链接","FriendLink","Friendlink","friends","FRIENDS",'Friends')if str in html_result.text]:
            bsObj = BeautifulSoup(html_result.text, 'html.parser')
            searchs=['section','aside','div']
            for search in searchs:
                sections = bsObj.find_all(search)
                for section in sections:
                    if [True for str in ("友链","友情链接","FriendLink","Friendlink","friends",'FRIENDS','Friends')if str in section.text]:
                        #print("在{}发现主页{}的友情链接".format(search,url))
                        a_urls = section.find_all('a')
                        
                        friend_link_url={}
                        for a_url in a_urls:
                            #print(a_url.get('href'))
                            if a_url.get('href')!=None:
                                ext=tldextract.extract(a_url.get('href'))
                                main_domain=ext.domain+'.'+ext.suffix
                                if re.search('(http)?(.+?)(com|net|blog|pw|cc|xyz|club|org|cn|io|info|me|im|win|link)(\/)?$',a_url.get('href')) != None and main_domain not in url:
                                    #print(a_url.text,(a_url.get('href').strip('/')))
                                    if a_url.get('href')!='http://www.52bug.cn':
                                        friend_link_url[a_url.text]=(a_url.get('href').strip('/'))
                                        status=True
                            
                        
                        if friend_link_url:
                            result.append(friend_link_url)
                if len(result)>1:
                    #取交集       
                    #print("友链出现在多个同样标签中，请手工验证{}".format(url))
                    #print(result)
                    #no_link.pop(key)
                    return result[-1],key
                    
                if status==True:
                    break
            if status==False:
                #print('未在已有标签中发现{}友链，可能存在于其他标签中,请手工验证'.format(url))
                pass
            #no_link.pop(key)
        else:
            #print('未发现友链关键字%s' %url)
            pass
       
    except Exception as e:
        #print("{}{}".format(url,e))
        pass
        
    
    return friend_link_url,key
def find_security_blog(url_dict):
    security_dict={}
    no_security_dict={}
    s = requests.session()
    s.keep_alive = False
    s.adapters.DEFAULT_RETRIES = 10
    for key in url_dict.keys():
        try:
            if not url_dict[key].startswith("http"):
                url_dict[key]="http://"+url_dict[key]
            #print(key,url_dict[key],s.get(url_dict[key],timeout=10))
            html=s.get(url_dict[key],timeout=10)
            html.encoding="utf-8"
            if html.status_code==200:
                if [True for str in ("二进制安全","安全攻防","安全数据","安全运营","XSS","信息安全协会","代码审计","网络安全","security","Penetration",'渗透','CTF','CSRF','SQL注入','RCE','CVE','漏洞分析','安全研究')if str in html.text]:
                    security_dict[key]=url_dict[key]
                else:
                    no_security_dict[key]=url_dict[key]
                    
            else:
                #print("网页暂时打不开")
                pass
        except Exception as e:
            #print("{}{}".format(e,url_dict[key]))
            pass
            no_security_dict[key]=url_dict[key]  
    return security_dict,no_security_dict
#search_link('xjj','http://www.yqxiaojunjie.com/')
#test
#print(from_link_search_friend(from_index_search_links('http://balis0ng.com')))
#print(from_index_search_links('http://le4f.net'))
#print(from_index_search_links('http://leavesongs.com'))
#print(from_index_search_links('http://www.venenof.com'))
#search_link('f','http://wooyaa.me/')
#find_security_blog({'1': 'https://www.youngxj.cn/'})

In [4]:
friends={}
result={}
dict={}
dict[init_url]=from_link_search_friend(init_url)
friends['L-team']=dict
print('now %s' %friends)

now {'L-team': {'http://l-team.org/about-us.html': {'tools': 'http://tool.leavesongs.com', 'Da2din9o': 'http://dazdingo.net', 'Zing': 'http://z1ng.net', 'DM_': 'http://x0day.me', 'Phiti0n': 'http://leavesongs.com', 'Le4F': 'http://le4f.net', 'Kuuki': 'http://blog.esu.im', 'Bigtang': 'http://bigtang.org', 'Chu': 'http://sh3ll.me', 'Cyrils': 'http://www.cyrils.org', 'F0r': 'http://www.f0r.info', 'Think': 'http://th1nk.info', 'math1as': 'http://www.math1as.com', 'rj1ng': 'http://rj1ng.com', 'Silver': 'https://www.iret.xyz', 'Sud0': 'http://sudalover.com', '几何_me7ell': 'http://blog.7ell.me', 'Ricky': 'https://rickyhao.com', 'Omego': 'http:/xd-a8.com', '1phan': 'http://1phan.cc', 'grt1st': 'http://www.grt1st.cn', 'Klaus': 'http://klaus.link', '天上的因幡': 'http://rabbithouse.me', 'skye': 'http://sky3.pw'}}}


In [ ]:
#自身去重，与上次对比去重，得到下步需要探测的字典
while(True):
    next={}
    for i in friends.keys():
        for j in friends[i].keys():
            for k in friends[i][j].keys():
                next[k]=friends[i][j][k]
    next={next[key]:key for key in next}
    next={next[key]:key for key in next}
    #print(next)
    netodel=[]
    for i in result.keys():
        for j in result[i].keys():
            for k in next.keys():
                if j==next[k]:
                    netodel.append(next[k])
    netodel=list(set(netodel))
    next={next[key]:key for key in next}
    for i in netodel:
        next.pop(i)
    next={next[key]:key for key in next}
    #print(next)
    security_host,no_security_host=find_security_blog(next)
    #初始化结果字典集
    for i in security_host.keys():
        #print(friends[i][j][k])
        dict={}
        dict[security_host[i]]=''
        result[i]=dict
    #print(result)
    
    #主机存活探测
    """
    for key in friends.keys():
        for key2 in friends[key].keys():
            sur_host,problem_host,fail_host=get_host(friends[key][key2])
    #注册未存活的主机和不正常的网页
    for i in problem_host.keys():
        for j in result[i].keys():
            result[i][j]='404notfound'
    for i in fail_host.keys():
        for j in result[i].keys():
            result[i][j]='nohost'
    """
    #security_host,no_security_host=find_security_blog(next)
    #第一遍筛选
    no_link=security_host.copy()
    friends={}
    for key in security_host.keys():
        #print('%s' %security_host[key])   
        friend_link=from_index_search_links(security_host[key])
        dict={}
        if friend_link:
            no_link.pop(key)
            #print('%s found ' %security_host[key])
            dict[security_host[key]]=from_link_search_friend(friend_link)
            friends[key]=dict

        else:
            #print('%s not found ' %security_host[key])
            pass

    #print("第一次搜索结果：{}".format(len(friends)))
    #第二遍筛选
    needtodel=[]
    for i in no_link.keys():
        friend_link,key=search_link(i,no_link[i])
        if not friend_link:
            pass
        else:
            #print(friend_link,key)
            dict={}
            dict[no_link[i]]=friend_link
            friends[i]=dict
            needtodel.append(key)
    for i in needtodel:
        no_link.pop(i)
    #print("第二次搜索结果：{}".format(len(friends)))
    #赋值给最终的结果集
    for key in friends.keys():
        result[key]=friends[key]
    #扫尾注册
    for i in no_link.keys():
        dict={}
        dict[no_link[i]]='nolink'
        result[i]=dict
    print("最终结果集为{}".format(result))
    print(len(result))

最终结果集为{'Phiti0n': {'http://leavesongs.com': {'http://xdsec.org/': 'http://xdsec.org', 'http://www.polaris-lab.com/': 'http://www.polaris-lab.com', 'http://le4f.net/': 'http://le4f.net', 'http://bigtang.org/': 'http://bigtang.org', 'http://sh3ll.me/': 'http://sh3ll.me', 'https://blog.flanker017.me': 'https://blog.flanker017.me', 'http://quininer.github.io/': 'http://quininer.github.io', 'http://blog.wils0n.cn/': 'http://blog.wils0n.cn', 'http://iret.xyz/': 'http://iret.xyz', 'http://appleu0.sinaapp.com/': 'http://appleu0.sinaapp.com', 'http://wrfly.kfd.me/': 'http://wrfly.kfd.me', 'http://wolvez.club/': 'http://wolvez.club', 'http://www.jmpoep.com/': 'http://www.jmpoep.com', 'http://www.expshell.com/': 'http://www.expshell.com', 'http://www.hackersb.cn/': 'http://www.hackersb.cn', 'https://imlonghao.com/': 'https://imlonghao.com', 'https://phpinfo.me/': 'https://phpinfo.me', 'http://www.cnbraid.com/': 'http://www.cnbraid.com', 'https://www.jisec.com/': 'https://www.jisec.com', 'https://